In [1]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt, style; style.use('fivethirtyeight')
import seaborn as sns; sns.set(context='talk')

from IPython.display import display, HTML, IFrame

Dataset Exploration

In [2]:
df = pd.read_csv('./data/twtc.csv')

df_label_mask = (df.drop(columns='label') == 0).assign(label=False)
df = df.mask(df_label_mask)

df.describe().T[['mean', 'std', 'min', '50%', 'max']]

,mean,std,min,50%,max
Unnamed: 0,4587.500000,2648.450018,1.00,4587.5,9174.00
age,20.810431,2.307676,15.30,21.0,31.90
year,2016.715095,1.979656,2013.00,2017.0,2019.00
eta,2018.802799,2.422862,2013.00,2019.0,2025.00
Arm,53.824127,6.897962,30.00,55.0,80.00
Changeup,49.858290,5.364976,30.00,50.0,70.00
Control,49.205290,5.059892,30.00,50.0,70.00
Curveball,52.929583,5.675287,35.00,55.0,70.00
Cutter,52.475000,4.936723,40.00,50.0,70.00
Fastball,59.531873,6.665786,40.00,60.0,80.00


In [4]:
pip install datasets

  Using cached datasets-1.18.3-py3-none-any.whl (311 kB)
  Using cached aiohttp-3.8.1-cp39-cp39-macosx_10_9_x86_64.whl (574 kB)
  Using cached dill-0.3.4-py2.py3-none-any.whl (86 kB)
  Using cached pyarrow-7.0.0-cp39-cp39-macosx_10_13_x86_64.whl (20.2 MB)
  Using cached xxhash-3.0.0-cp39-cp39-macosx_10_9_x86_64.whl (34 kB)
  Using cached multiprocess-0.70.12.2-py39-none-any.whl (128 kB)
  Using cached aiosignal-1.2.0-py3-none-any.whl (8.2 kB)
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)
  Using cached multidict-6.0.2-cp39-cp39-macosx_10_9_x86_64.whl (28 kB)
  Using cached yarl-1.7.2-cp39-cp39-macosx_10_9_x86_64.whl (121 kB)
  Using cached frozenlist-1.3.0-cp39-cp39-macosx_10_9_x86_64.whl (36 kB)
Note: you may need to restart the kernel to use updated packages.


In [68]:
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from transformers import TrainingArguments, Trainer
from transformers import BertForSequenceClassification, BertTokenizer

In [12]:
labeled = pd.read_csv('data/labeled_scouting.csv')
test = pd.read_csv('data/augment/test.csv')
train = pd.concat([labeled, test]).drop_duplicates(keep=False)

custom_dataset_train = Dataset.from_pandas(train)
custom_dataset_test = Dataset.from_pandas(test)
custom_dataset_train

Dataset({
    features: ['text', 'label', '__index_level_0__'],
    num_rows: 6222
})

In [13]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [41]:
train = custom_dataset_train.map(lambda e: tokenizer(e['text'], truncation=True, padding="max_length"), batched=True)
train = train.remove_columns(["text", "__index_level_0__"])
test = custom_dataset_test.map(lambda e: tokenizer(e['text'], truncation=True, padding="max_length"), batched=True)
test = test.remove_columns(["text"])

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [42]:
pd.DataFrame(train)

,label,input_ids,token_type_ids,attention_mask
0,0,"[101, 2711, 2003, 1037, 2504, 1015, 3348, 2504...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,1,"[101, 1996, 3029, 2031, 3734, 2711, 3807, 1010...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,1,"[101, 2772, 2005, 2019, 2682, 1011, 10453, 100...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,1,"[101, 2009, 2411, 3138, 2051, 2005, 2216, 2152...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,1,"[101, 2772, 2005, 2019, 2682, 1011, 10453, 100...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...,...
6217,0,"[101, 2711, 2275, 1037, 5785, 2501, 2007, 2403...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
6218,0,"[101, 2711, 2275, 1996, 2711, 2501, 2005, 1316...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
6219,0,"[101, 2004, 1037, 13758, 1010, 1999, 2286, 101...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
6220,1,"[101, 2178, 9824, 1999, 1996, 3029, 14655, 101...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [76]:
def mixup(row1, row2, label=1, l=0.5):
    mixed = list(map(int, l * np.array(row1['input_ids'].iloc[0]) + (1 - l) * np.array(row2['input_ids'].iloc[0])))
    if sum(row1['attention_mask'].iloc[0]) > sum(row2['attention_mask'].iloc[0]):
        new_attention = row1['attention_mask'].iloc[0]
    else:
        new_attention = row2['attention_mask'].iloc[0]

    return pd.DataFrame([{
        'label': label,
        'input_ids': mixed,
        'token_type_ids': row1['token_type_ids'].iloc[0],
        'attention_mask': new_attention
    }])

In [78]:
tokenized_train = pd.DataFrame(train)
pos = tokenized_train[tokenized_train["label"] == 1].copy(deep=True)
neg = tokenized_train[tokenized_train["label"] == 0].copy(deep=True)
for i in range(len(neg) - len(pos)):
    row1 = pos.sample()
    row2 = pos.sample()
    new_embed = mixup(row1, row2)
    tokenized_train = tokenized_train.append(new_embed, ignore_index=True)
shuffle(tokenized_train)
tokenized_train = Dataset.from_pandas(tokenized_train)

   label                                          input_ids  \
0      1  [101, 2353, 1021, 1030, 1769, 9493, 4293, 2047...   

                                      token_type_ids  \
0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   

                                      attention_mask  
0  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  


In [10]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

arguments = TrainingArguments(
    output_dir="sample_SD_trainer",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy="epoch",
    learning_rate=2e-5,
    load_best_model_at_end=True,
    seed=224
)


def compute_metrics(eval_pred):
    """Called at the end of validation. Gives accuracy"""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # calculates the accuracy
    return {"accuracy": np.mean(predictions == labels)}


trainer = Trainer(
    model=model,
    args=arguments,
    train_dataset=train,
    eval_dataset=test, # change to test when you do your final evaluation!
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [11]:
from transformers import TrainerCallback, EarlyStoppingCallback

class LoggingCallback(TrainerCallback):
    def __init__(self, log_path):
        self.log_path = log_path
        
    def on_log(self, args, state, control, logs=None, **kwargs):
        _ = logs.pop("total_flos", None)
        if state.is_local_process_zero:
            with open(self.log_path, "a") as f:
                f.write(json.dumps(logs) + "\n")


trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=1, early_stopping_threshold=0.0))
trainer.add_callback(LoggingCallback("sample_SD_trainer/log.jsonl"))

In [ ]:
trainer.train()

/Users/amolsingh/anaconda3/envs/local_nmt/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 6222
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1167


Epoch,Training Loss,Validation Loss


From_Pretrained

In [ ]:
# To load our saved model, we can pass the path to the checkpoint into the `from_pretrained` method:
test_str = "I enjoyed the movie!"

finetuned_model = AutoModelForSequenceClassification.from_pretrained("sample_SD_trainer/checkpoint-24")
model_inputs = tokenizer(test_str, return_tensors="pt")
prediction = torch.argmax(finetuned_model(**model_inputs).logits)
print(["NEGATIVE", "POSITIVE"][prediction])